# Redes Neuronales Artificales

# Importación de librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importar el data set

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [4]:
y

array([1, 0, 1, ..., 1, 1, 0])

# Parte 1 - Pre procesado de datos

# Codificar datos categóricos

In [5]:
from sklearn.preprocessing import LabelEncoder

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers = [
        ('Churn_Modelling', # Un nombre de la transformación
        OneHotEncoder(categories='auto'), # La clase a la que transformar
        [1] # Las columnas a transformar
        )
    ], remainder='passthrough'                        
)

X = transformer.fit_transform(X)
X = X[:, 1:]
X

array([[0.0, 0.0, 619, ..., 1, 1, 101348.88],
       [0.0, 1.0, 608, ..., 0, 1, 112542.58],
       [0.0, 0.0, 502, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 709, ..., 0, 1, 42085.58],
       [1.0, 0.0, 772, ..., 1, 0, 92888.52],
       [0.0, 0.0, 792, ..., 1, 0, 38190.78]], dtype=object)

# Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Escalado de variables

In [8]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Parte 2 - Construir la RNA

# Importar Keras y librerías adicionales


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense

# Inicializar la RNA

In [10]:
classifier = Sequential()

# Añadir las capas de entrada y primera capa oculta


In [11]:
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 11))

# Añadir la segunda capa oculta

In [12]:
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

# Añadir la capa de salida

In [13]:
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

# Compilar la RNA

In [14]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# Ajustamos la RNA al Conjunto de Entrenamiento

In [15]:
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 5s 3ms/step - loss: 0.5557 - accuracy: 0.7960
Epoch 2/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4291 - accuracy: 0.7980
Epoch 3/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4306 - accuracy: 0.7944
Epoch 4/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4153 - accuracy: 0.8242
Epoch 5/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4265 - accuracy: 0.8259
Epoch 6/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4154 - accuracy: 0.8337
Epoch 7/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4062 - accuracy: 0.8336
Epoch 8/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4150 - accuracy: 0.8350
Epoch 9/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3980 - accuracy: 0.8374
Epoch 10/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4163 - accura

# Parte 3 - Evaluar el modelo y calcular predicciones finales

# Predicción de los resultados con el Conjunto de Testing

In [16]:
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5)

Utiliza nuestro modelo de RNA para predecir si el cliente con la siguiente información abandonará el banco: <br>
* Geografia: Francia<br>
* Puntaje de crédito: 600<br>
* Género masculino<br>
* Edad: 40 años de edad<br>
* Tenencia: 3 años.<br>
* Saldo: 60000 dólares <br>
* Número de productos: 2 <br>
* ¿Este cliente tiene una tarjeta de crédito? Sí<br>
* ¿Es este cliente un miembro activo? Sí<br>
* Salario estimado: 50000 dólares<br>
<br>
Entonces, ¿deberíamos decir adiós a ese cliente?<br>

Notas:<br>
Francia  es 0, 0<br>
España   es 0, 1<br>
Alemania es 1, 0<br>
Hombre es 1<br>
Mujer es 0

In [17]:
new_prediction = classifier.predict(sc_X.transform(np.array([[0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
print(new_prediction)
print(new_prediction > 0.5)

[[0.08452561]]
[[False]]


No, no debemos decir adiós a este cliente dado que su score es inferior a 0.5

# Elaborar una matriz de confusión

In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1545,   50],
       [ 269,  136]])

# Evaluar el resultado

In [19]:
print((cm[0][0]+cm[1][1]) / cm.sum())

0.8405


## Parte 4 - Evaluar, mejorar y Ajustar la RNA


### Evaluar la **RNA**


In [10]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [11]:
def build_classifier():
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
  classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

In [12]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, nb_epoch = 100)
accuracies = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv = 10, n_jobs=-1, verbose = 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   33.2s finished


In [13]:
mean = accuracies.mean()
variance = accuracies.std()
print("Mean:", mean)
print("Variance:", variance)

### Mejorar la RNA

#### Regularización de Dropout para evitar el *overfitting*


### Ajustar la *RNA*


In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
def build_classifier(optimizer):
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu", input_dim = 11))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))
  classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier


In [16]:
classifier = KerasClassifier(build_fn = build_classifier)

In [20]:
parameters = {
    'batch_size': [15, 20, 25],
    'nb_epoch': [400, 500, 600], 
    'optimizer': ['adam', 'rmsprop']
}

In [21]:
grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', 
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

534/534 [==============================] - 2s 2ms/step - loss: 0.7427 - accuracy: 0.7958


In [22]:
best_parameters, best_accuracy

({'activation': 'relu',
  'batch_size': 15,
  'nb_epoch': 500,
  'optimizer': 'adam'},
 0.805875)